In [1]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

Loading BokehJS ...

In [2]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')

In [3]:
postflash_data[0:10]

,filename,propid,path,obsdate,obstime,shutter,flash_lvl,flash_dur,flash_cur,subarray,exp_time,mean,sig_mean,median,sig_median,std,sig_std,norm_by_dur,datetime
rootname,,,,,,,,,,,,,,,,,,,
ic6de2u9q,ic6de2u9q_flc.fits,13078,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2013-03-09,13:59:17,B,12,4.7,LOW,True,0.5,7.312437,7.048593,6.917119,6.917119,17.943027,2.911503,1.499701,2013-03-09 13:59:17
icha24imq,icha24imq_flc.fits,13560,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2014-06-29,04:16:36,A,3,1.2,LOW,True,0.5,2.170188,1.976714,1.933412,1.933412,15.109935,2.360522,1.647262,2014-06-29 04:16:36
idp530aiq,idp530aiq_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-06-22,04:55:12,B,15,0.2,MED,True,0.5,8.972477,8.680973,8.854218,8.854218,22.985115,3.162331,43.404864,2018-06-22 04:55:12
ieg845feq,ieg845feq_flc.fits,16398,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2020-12-14,20:55:49,B,7310,100.0,MED,False,0.5,4685.206055,4683.406877,4759.056641,4759.056641,436.754791,419.022524,46.834069,2020-12-14 20:55:49
icrz19f7q,icrz19f7q_flc.fits,14006,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2015-05-25,22:31:24,B,12,4.7,LOW,True,0.5,7.337009,7.018181,6.969007,6.969007,27.715347,2.952669,1.493230,2015-05-25 22:31:24
idus35y6q,idus35y6q_flc.fits,15573,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-08-15,10:09:26,A,7310,100.0,MED,False,0.5,4876.872559,4875.340172,4947.657715,4947.657715,436.621338,430.077256,48.753402,2019-08-15 10:09:26
ie1b44diq,ie1b44diq_flc.fits,15718,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2019-12-09,09:02:57,B,22,0.3,MED,False,100.0,16.624331,14.670476,14.623810,14.623810,78.962532,3.976113,48.901587,2019-12-09 09:02:57
idp531a7q,idp531a7q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2018-07-20,04:24:43,B,12,4.7,LOW,True,0.5,7.372123,7.059274,6.966012,6.966012,18.031178,2.999503,1.501973,2018-07-20 04:24:43
idp543c4q,idp543c4q_flc.fits,14984,/grp/hst/wfc3u/final_postflash_2021/mastDownlo...,2017-12-11,05:29:33,B,12,4.7,LOW,False,0.5,8.491176,7.938883,7.974644,7.974644,52.009426,3.177174,1.689124,2017-12-11 05:29:33


In [4]:
low_sub_pf_A_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 3)]

In [5]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_3['obsdate'] + ' ' + low_sub_pf_A_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_3['norm_by_dur'])
y = low_sub_pf_A_lvl_3['norm_by_dur']/low_sub_pf_A_lvl_3['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 3 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

-3.4492398520047405e-06


In [32]:
low_sub_pf_B_lvl_3 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 3)]

In [33]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_3['obsdate'] + ' ' + low_sub_pf_B_lvl_3['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_3['norm_by_dur'])
y = low_sub_pf_B_lvl_3['norm_by_dur']/low_sub_pf_B_lvl_3['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 3 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -3.5259739029225614e-06


In [34]:
low_sub_pf_A_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 5)]

# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_5['obsdate'] + ' ' + low_sub_pf_A_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_5['norm_by_dur'])
y = low_sub_pf_A_lvl_5['norm_by_dur']/low_sub_pf_A_lvl_5['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 5 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -4.553392164842169e-06


In [35]:
low_sub_pf_B_lvl_5 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 5)]
low_sub_pf_A_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_B_lvl_8 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 8)]
low_sub_pf_A_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_B_lvl_10 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 10)]
low_sub_pf_A_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_B_lvl_12 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 12)]
low_sub_pf_A_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_lvl'] == 20)]
low_sub_pf_B_lvl_20 = postflash_data.loc[(postflash_data['flash_cur'] == 'LOW') & (postflash_data['subarray'] == True) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_lvl'] == 20)]


In [36]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_5['obsdate'] + ' ' + low_sub_pf_B_lvl_5['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_5['norm_by_dur'])
y = low_sub_pf_B_lvl_5['norm_by_dur']/low_sub_pf_B_lvl_5['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 5 Shutter B")
p.line(x_datetime,y_pred,color='red', name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -2.1859381538526042e-06


In [37]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_8['obsdate'] + ' ' + low_sub_pf_A_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_8['norm_by_dur'])
y = low_sub_pf_A_lvl_8['norm_by_dur']/low_sub_pf_A_lvl_8['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 8 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -4.873644632536405e-06


In [38]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_8['obsdate'] + ' ' + low_sub_pf_B_lvl_8['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_8['norm_by_dur'])
y = low_sub_pf_B_lvl_8['norm_by_dur']/low_sub_pf_B_lvl_8['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 8 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -3.133432353211278e-06


In [39]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_10['obsdate'] + ' ' + low_sub_pf_A_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_10['norm_by_dur'])
y = low_sub_pf_A_lvl_10['norm_by_dur']/low_sub_pf_A_lvl_10['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 10 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -3.4494660409023584e-06


In [40]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_10['obsdate'] + ' ' + low_sub_pf_B_lvl_10['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_10['norm_by_dur'])
y = low_sub_pf_B_lvl_10['norm_by_dur']/low_sub_pf_B_lvl_10['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 10 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.148954204414007e-06


In [41]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_12['obsdate'] + ' ' + low_sub_pf_A_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_12['norm_by_dur'])
y = low_sub_pf_A_lvl_12['norm_by_dur']/low_sub_pf_A_lvl_12['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 12 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.541085457254137e-06


In [42]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_12['obsdate'] + ' ' + low_sub_pf_B_lvl_12['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_12['norm_by_dur'])
y = low_sub_pf_B_lvl_12['norm_by_dur']/low_sub_pf_B_lvl_12['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.1)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 12 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.69377315473145e-06


In [43]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_A_lvl_20['obsdate'] + ' ' + low_sub_pf_A_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_A_lvl_20['norm_by_dur'])
y = low_sub_pf_A_lvl_20['norm_by_dur']/low_sub_pf_A_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 20 Shutter A")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -5.099668467994132e-06


In [44]:
# normalized to the mean of first observation
x_datetime = pd.to_datetime(low_sub_pf_B_lvl_20['obsdate'] + ' ' + low_sub_pf_B_lvl_20['obstime'])
sorted_x = x_datetime.sort_values()
first_rootname = sorted_x.index[0]
x = Time(x_datetime,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(low_sub_pf_B_lvl_20['norm_by_dur'])
y = low_sub_pf_B_lvl_20['norm_by_dur']/low_sub_pf_B_lvl_20['norm_by_dur'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print("Slope: ", popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.95, 1.05)
p.circle(x_datetime, y,size = 4, name = "Low Subarray Lvl 20 Shutter B")
p.line(x_datetime,y_pred,color='red',name='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

Slope:  -4.999002239421393e-06
